In [ ]:
# RNN は入出力シーケンスによっていくつか分類される

# シーケンスシーケンス(同時に入出力を受け取る)：株価など時系列予測。N日までの株価を与えるとN-1日から～明日までの予測を出す
# シーケンスベクトル(時系列最後以外出力を無視する)：映画の単語シーケンスを与えるとスコア(-1～1とか)を出す
# ベクトルシーケンス(最初の入力以外のタイムステップは0を入力)：ベクトルをシーケンスに変換する。イメージを入力しイメージのタイトルを出す
# 遅延シーケンスシーケンス(エンコーダデコーダ)：翻訳など。ある言語の文章を単一ベクトルに変換し、そのベクトルを別の言語に展開する

In [2]:
# お決まり設定
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [ ]:
# TensorFlowによるRNN
# 

In [3]:
import tensorflow as tf
reset_graph()

n_inputs = 3 # 入力となる特徴量数
n_neurons = 5 #　再帰ニューロン数

X0 = tf.placeholder(tf.float32, [None, n_inputs]) # batch_size
X1 = tf.placeholder(tf.float32, [None, n_inputs]) # input_size

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
# tf.nn.rnn_cell.BasicRNNCell() セルのコピーを作って(タイムステップごとに一つずつ)アンロールされたRNNを構築する。だからニューロン数が引数
# デフォルトの活性化関数はtanhなど他にもパラメータ有

output_seqs, states = tf.nn.static_rnn(basic_cell, [X0, X1],
                                       dtype=tf.float32)
# tf.nn.static_rnn() セル(再帰ニューロン)を連鎖させてアンロールされたRNNネットワークを作る
# RNNCellのインスタンス：今回はbasic_cell
# inputs：入力の長さTのリスト、それぞれTensor形状 [batch_size, input_size]、またはそのような要素のネストしたタプル。今回は[X0,X1]
# 戻り値は個々のタイムステップの出力テンソルを一つにまとめたリスト、ネットワークの最終結果を格納するテンソル

Y0, Y1 = output_seqs
# 個々のタイムステップの出力テンソル定義をY0,Y1に格納


init = tf.global_variables_initializer()

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.


In [4]:
import numpy as np

X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]])
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]])

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [1]:
from tensorboard import show_graph

In [5]:
show_graph(tf.get_default_graph())

In [6]:
Y0_val # T=1の時点での出力

array([[ 0.30741334, -0.32884315, -0.6542847 , -0.9385059 ,  0.52089024],
       [ 0.99122757, -0.9542541 , -0.7518079 , -0.9995208 ,  0.9820235 ],
       [ 0.9999268 , -0.99783254, -0.8247353 , -0.9999963 ,  0.99947774],
       [ 0.996771  , -0.68750614,  0.8419969 ,  0.9303911 ,  0.8120684 ]],
      dtype=float32)

In [7]:
Y1_val # T=1の時点での出力

array([[ 0.99998885, -0.99976057, -0.0667929 , -0.9999803 ,  0.99982214],
       [-0.6524943 , -0.51520866, -0.37968948, -0.5922594 , -0.08968379],
       [ 0.99862397, -0.99715203, -0.03308626, -0.9991566 ,  0.9932902 ],
       [ 0.99681675, -0.9598194 ,  0.39660627, -0.8307606 ,  0.79671973]],
      dtype=float32)

In [12]:
states

<tf.Tensor 'rnn/basic_rnn_cell/Tanh_1:0' shape=(?, 5) dtype=float32>

In [ ]:
# 上記のやり方ではタイムステップごとに入力プレースホルダーと出力テンソルの定義が必要になる。
# 実行時にもタイムステップ数分、入力に送り込み、出力操作が必要。
# 上記を解決するため、動的なアンロールを行う dynamic.run() 関数を使う

In [43]:
n_steps = 3 # タイムステップ数　1SIDあたりの時系列が３件あるイメージ
n_inputs = 4 # インプットの特徴量数 上記3件データにおける1件当たりの項目数
n_neurons = 5 # 再帰ニューロン数 記憶セルがどれくらい大きいかを定義

In [44]:
reset_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
# タイムステップのすべての入力をこれで定義

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
# tf.nn.dynamic_rnn :  [None, n_steps, n_inputs]という形状のテンソルに出力する

In [45]:
init = tf.global_variables_initializer()

In [46]:
X_batch = np.array([
        [[0, 1, 2, 3], [9, 8, 7, 6], [9, 8, 7, 6]],# instance 1
        [[3, 4, 5, 6], [0, 0, 0, 0], [0, 0, 0, 1]],# instance 2
        [[6, 7, 8, 9], [6, 5, 4, 3], [6, 5, 4, 3]],# instance 3
        [[9, 0, 1, 2], [3, 2, 1, 0], [3, 2, 1, 0]],# instance 4
#        [[8, 5, 4, 3], [6, 7, 8, 9], [6, 7, 8, 9]],# instance 5(add)
    ])

with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

In [20]:
print(outputs_val)

[[[-0.85115266  0.87358344  0.5802911   0.8954789  -0.0557505 ]
  [-0.999996    0.99999577  0.9981815   1.          0.37679607]]

 [[-0.9983293   0.9992038   0.98071456  0.999985    0.25192663]
  [-0.7081804  -0.0772338  -0.85227895  0.5845349  -0.78780943]]

 [[-0.9999827   0.99999535  0.9992863   1.          0.5159072 ]
  [-0.9993956   0.9984095   0.83422637  0.99999976 -0.47325212]]

 [[ 0.87888587  0.07356028  0.97216916  0.9998546  -0.7351168 ]
  [-0.9134514   0.3600957   0.7624866   0.99817705  0.80142   ]]

 [[-0.99433464  0.99877965  0.9954959   0.9999998   0.482667  ]
  [-0.9999963   0.999993    0.9879236   1.         -0.5406112 ]]]


In [18]:
show_graph(tf.get_default_graph())

In [ ]:
# 可変長入力の扱い
# dynamic_run()に渡す引数に工夫が必要

In [77]:
n_steps = 2
n_inputs = 3
n_neurons = 5

reset_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)

In [78]:
seq_length = tf.placeholder(tf.int32, [None])
# sequence_length 引数の作成。インスタンスごとに入力シーケンスの長さを渡すようにする
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32,
                                    sequence_length=seq_length)

In [79]:
init = tf.global_variables_initializer()

In [84]:
# この例では２個目の入力シーケンスには１個のインスタンスしかない
# この場合は入力テンソル X の定義に合わせて0ベクトルでパディングする必要がある

X_batch = np.array([
        # step 0     step 1
        [[0, 0, 2], [8, 0, 0]], # instance 1
        [[0, 4, 0], [0, 0, 0]], # instance 2 (padded with zero vectors)
        [[2, 0, 0], [0, 6, 0]], # instance 3
        [[0, 0, 2], [0, 0, 4]], # instance 4
    ])
seq_length_batch = np.array([2, 1, 2, 2])
# 実際のシーケンス当たりのインスタンス数を定義

In [85]:
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run(
        [outputs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})
    # 実行に渡すseq_length_batchを修正

In [86]:
print(outputs_val)
# 第二インスタンスの第二タイムステップの値が0ベクトルになっている
# outputs_val には個々のタイムステップの出力を出しているためこおうなる

[[[-0.84171796  0.4719534   0.14381935 -0.6426326   0.6411661 ]
  [-0.9808252   0.98558617 -0.71904945  0.9997032   0.9955954 ]]

 [[-0.8501176  -0.88174516  0.9581016   0.88445336 -0.9754706 ]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.29965404  0.5433119  -0.09989461  0.7027565   0.68391067]
  [-0.9485175  -0.89504844  0.9928569   0.975321   -0.99846804]]

 [[-0.84171796  0.4719534   0.14381935 -0.6426326   0.6411661 ]
  [-0.9983055   0.7827886  -0.21185385 -0.5428824   0.84333956]]]


In [87]:
print(states_val)
# 各セルの最終状態が格納。２個目のインスタンスは１個しかデータがないため、上記の１個目と同様になる。
# ２個目以外のインスタンスは２個データがあるので上記の２個目と同様になる。

[[-0.9808252   0.98558617 -0.71904945  0.9997032   0.9955954 ]
 [-0.8501176  -0.88174516  0.9581016   0.88445336 -0.9754706 ]
 [-0.9485175  -0.89504844  0.9928569   0.975321   -0.99846804]
 [-0.9983055   0.7827886  -0.21185385 -0.5428824   0.84333956]]
